Local Model(lama3.2:3b) Connection and Testing

In [106]:
from langchain_ollama import ChatOllama 
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough 
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.tools import tool


llm = ChatOllama(model='llama3.2:3b', base_url='http://localhost:11434',temperature=0.7)
llm.invoke('hi')

AIMessage(content='How can I assist you today?', additional_kwargs={}, response_metadata={'model': 'llama3.2:3b', 'created_at': '2025-02-25T21:09:30.98602Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3944842125, 'load_duration': 26438167, 'prompt_eval_count': 26, 'prompt_eval_duration': 3691000000, 'eval_count': 8, 'eval_duration': 217000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-f00d325b-97ee-4aa7-aa8f-d5d59ab80d41-0', usage_metadata={'input_tokens': 26, 'output_tokens': 8, 'total_tokens': 34})

Database Connection and Testing

In [107]:
db_user = "root"
db_password = "rootroot"
db_host = "localhost"
db_port = 3306
db_name = "atliq_tshirts"

from langchain_community.utilities.sql_database import SQLDatabase
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}",sample_rows_in_table_info=3)

print(db.dialect)
print(db.get_table_names())
print(db.table_info)
db.run("SELECT * FROM t_shirts")

mysql
['discounts', 't_shirts']

CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from t_shirts table:
t_shirt

"[(1, 'Nike', 'Black', 'XL', 48, 84), (2, 'Levi', 'White', 'XL', 35, 44), (3, 'Van Huesen', 'Blue', 'XS', 33, 71), (4, 'Nike', 'Red', 'S', 29, 66), (5, 'Nike', 'Black', 'S', 33, 89), (6, 'Nike', 'Black', 'M', 15, 44), (7, 'Levi', 'Blue', 'XS', 18, 97), (8, 'Van Huesen', 'Red', 'M', 36, 62), (9, 'Adidas', 'Red', 'XL', 23, 73), (10, 'Nike', 'Black', 'XS', 39, 43), (12, 'Nike', 'Blue', 'XL', 39, 72), (13, 'Levi', 'Black', 'L', 33, 97), (14, 'Van Huesen', 'Black', 'L', 40, 65), (15, 'Adidas', 'Blue', 'XL', 22, 14), (16, 'Levi', 'Black', 'S', 23, 90), (17, 'Levi', 'Blue', 'M', 35, 56), (18, 'Nike', 'White', 'M', 38, 15), (19, 'Van Huesen', 'White', 'S', 45, 69), (20, 'Nike', 'Blue', 'S', 41, 20), (21, 'Levi', 'Red', 'XS', 48, 39), (22, 'Adidas', 'Red', 'XS', 45, 28), (23, 'Levi', 'Black', 'XL', 42, 22), (24, 'Levi', 'Red', 'S', 17, 24), (25, 'Van Huesen', 'Black', 'XL', 25, 24), (28, 'Adidas', 'Red', 'S', 30, 90), (29, 'Adidas', 'White', 'M', 26, 26), (30, 'Nike', 'Red', 'L', 44, 26), (31, 

Creating SQL Query Chain

In [108]:
from langchain.chains import create_sql_query_chain

sql_chain = create_sql_query_chain(llm, db)

In [109]:
sql_chain.get_prompts()[0].pretty_print()

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the S

In [131]:
sql_chain

RunnableAssign(mapper={
  input: RunnableLambda(...),
  table_info: RunnableLambda(...)
})
| RunnableLambda(lambda x: {k: v for k, v in x.items() if k not in ('question', 'table_names_to_use')})
| PromptTemplate(input_variables=['input', 'table_info'], input_types={}, partial_variables={'top_k': '5'}, template='You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.\nPay attention to use only the column names you can see in the tables

Generating SQL Query

In [144]:
question = "list the brands that have discounts? You MUST RETURN ONLY MYSQL QUERIES."
response = sql_chain.invoke({'question': question})
print(response)

Answer: The brands that have discounts are 'Nike' and 'Levi'.

SQLQuery: 
```sql
SELECT brand FROM t_shirts JOIN discounts ON t_shirts.t_shirt_id = discounts.t_shirt_id WHERE discounts.pct_discount IS NOT NULL GROUP BY brand;
```


Calling the funion from askllm.py

In [138]:
from scripts.llm import ask_llm
from langchain_core.runnables import chain

In [139]:
@chain
def get_correct_sql_query(input):
    context = input['context']
    question = input['question']

    intruction = """
        Use above context to fetch the correct SQL query for following question
        {}

        Do not enclose query in ```sql and do not write preamble and explanation.
        You MUST return only single SQL query.
    """.format(question)

    response = ask_llm(context=context, question=intruction)

    return response


In [140]:
response = get_correct_sql_query.invoke({'context': response, 'question': question})

In [141]:
db.run(response)

'[(4,)]'

Final Chain

In [118]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

In [119]:
execute_query = QuerySQLDataBaseTool(db=db)
sql_query = create_sql_query_chain(llm, db)

final_chain = (
    {'context': sql_query, 'question': RunnablePassthrough()}
    | get_correct_sql_query
    | execute_query
)

In [129]:
question = "how many nike t-shirts are there in black color and size XL? You MUST RETURN ONLY MYSQL QUERIES."

response = final_chain.invoke({'question': question})
print(response)

[(1,)]


 MySQL Query Execution with Agents with LangGraph

In [123]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [124]:
tools = toolkit.get_tools()
tools

[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x11dab07a0>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x11dab07a0>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x11dab07a0>),
 QuerySQLCheckerTool(description='Use this tool to double check if your 

In [125]:
from langchain_core.messages import SystemMessage

SQL_PREFIX = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.


DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables."""

system_message = SystemMessage(content=SQL_PREFIX)

In [126]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools, state_modifier=system_message, debug=False)

In [127]:
question = "how many nike t-shirts are there in black color?"
# question = "How many departments are there?"

agent_executor.invoke({"messages": [HumanMessage(content=question)]})

{'messages': [HumanMessage(content='how many nike t-shirts are there in black color?', additional_kwargs={}, response_metadata={}, id='7338a70e-1d17-440d-a6bf-40d91b29f923'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:3b', 'created_at': '2025-02-25T21:12:18.334365Z', 'done': True, 'done_reason': 'stop', 'total_duration': 7173549958, 'load_duration': 38675167, 'prompt_eval_count': 721, 'prompt_eval_duration': 6249000000, 'eval_count': 27, 'eval_duration': 851000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-10bf1cc6-8b54-46bb-9f27-c26b0fd315a7-0', tool_calls=[{'name': 'sql_db_query_checker', 'args': {'query': 'SELECT COUNT(*) FROM products WHERE brand = '}, 'id': '99ca472c-2684-4123-a193-65b8030602db', 'type': 'tool_call'}], usage_metadata={'input_tokens': 721, 'output_tokens': 27, 'total_tokens': 748}),
  ToolMessage(content="SELECT COUNT(*) FROM products WHERE brand = 'Double'", name='sql_db_query_

In [130]:
for s in agent_executor.stream(
    {"messages": [HumanMessage(content=question)]}
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:3b', 'created_at': '2025-02-25T23:32:55.502265Z', 'done': True, 'done_reason': 'stop', 'total_duration': 16857208709, 'load_duration': 851718000, 'prompt_eval_count': 733, 'prompt_eval_duration': 12605000000, 'eval_count': 87, 'eval_duration': 3083000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-a81552d9-188e-4889-9314-3fd822ea7796-0', tool_calls=[{'name': 'sql_db_query_checker', 'args': {'query': 'SELECT COUNT(*) FROM shirt WHERE color = '}, 'id': '762079ee-b5f9-4dff-9356-4a8388d07786', 'type': 'tool_call'}, {'name': 'sql_db_query_checker', 'args': {'query': 'SELECT * FROM shirt WHERE color = '}, 'id': 'b750de3b-b977-4a9f-9874-b74d9aa2d819', 'type': 'tool_call'}, {'name': 'sql_db_list_tables', 'args': {}, 'id': '410c8fde-130d-44ed-90a9-5533821c6d19', 'type': 'tool_call'}, {'name': 'sql_db_schema', 'args': {'table_names': 'shirt'}, 'i